## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,busselton,AU,-33.6500,115.3333,58.05,57,70,12.46,broken clouds
1,1,rosetta,EG,31.4044,30.4164,78.62,64,1,15.52,clear sky
2,2,vaini,TO,-21.2000,-175.2000,80.76,83,20,8.05,few clouds
3,3,tasiilaq,GL,65.6145,-37.6368,26.26,62,24,6.67,few clouds
4,4,ushuaia,AR,-54.8000,-68.3000,35.26,69,75,5.75,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 25
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,busselton,AU,-33.6500,115.3333,58.05,57,70,12.46,broken clouds
3,3,tasiilaq,GL,65.6145,-37.6368,26.26,62,24,6.67,few clouds
4,4,ushuaia,AR,-54.8000,-68.3000,35.26,69,75,5.75,light snow
6,6,mutsu,JP,41.2894,141.2169,71.92,60,100,8.23,overcast clouds
7,7,kysyl-syr,RU,63.8986,122.7617,47.77,55,100,11.03,overcast clouds
8,8,souillac,MU,-20.5167,57.5167,70.36,94,75,3.44,broken clouds
9,9,vila franca do campo,PT,37.7167,-25.4333,68.41,89,92,4.18,overcast clouds
11,11,cape town,ZA,-33.9258,18.4232,55.89,73,0,5.01,clear sky
12,12,punta arenas,CL,-53.1500,-70.9167,35.71,100,90,4.61,overcast clouds
13,13,bulembu,SZ,-25.9667,31.1333,54.55,83,99,5.30,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                443
City                   443
Country                439
Lat                    443
Lng                    443
Max Temp               443
Humidity               443
Cloudiness             443
Wind Speed             443
Current Description    443
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,busselton,AU,58.05,broken clouds,-33.6500,115.3333,
3,tasiilaq,GL,26.26,few clouds,65.6145,-37.6368,
4,ushuaia,AR,35.26,light snow,-54.8000,-68.3000,
6,mutsu,JP,71.92,overcast clouds,41.2894,141.2169,
7,kysyl-syr,RU,47.77,overcast clouds,63.8986,122.7617,
8,souillac,MU,70.36,broken clouds,-20.5167,57.5167,
9,vila franca do campo,PT,68.41,overcast clouds,37.7167,-25.4333,
11,cape town,ZA,55.89,clear sky,-33.9258,18.4232,
12,punta arenas,CL,35.71,overcast clouds,-53.1500,-70.9167,
13,bulembu,SZ,54.55,overcast clouds,-25.9667,31.1333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")
        hotel_df.loc[index, "Hotel Name"] = np.nan

hotel_df

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not foun

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,busselton,AU,58.05,broken clouds,-33.6500,115.3333,Observatory Guest House
3,tasiilaq,GL,26.26,few clouds,65.6145,-37.6368,The Red House
4,ushuaia,AR,35.26,light snow,-54.8000,-68.3000,Albatros Hotel
6,mutsu,JP,71.92,overcast clouds,41.2894,141.2169,Mutsu Park Hotel
7,kysyl-syr,RU,47.77,overcast clouds,63.8986,122.7617,NaN
...,...,...,...,...,...,...,...
666,lunca cernii de jos,RO,59.79,overcast clouds,45.6333,22.6167,Hanul Lupilor
668,namwala,ZM,68.81,few clouds,-15.7504,26.4384,Corner Point Lodge
669,kingisepp,RU,35.89,overcast clouds,59.3733,28.6134,Narva
670,tokur,RU,48.38,scattered clouds,53.1333,132.9000,NaN


In [9]:
len(hotel_df)

439

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
len(clean_hotel_df)

386

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{Hotel Name}</dd>
<dt>Country</dt><dd>{City}</dd>
<dt>Current Weather</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))